<a href="https://colab.research.google.com/github/simonefmr/datascience_mvp/blob/main/MVP_desenvolvimento.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **MVP Pós Data Science - PUC Rio - Sprint 1**

## **1. Introdução**

O Banco Central do Brasil divulga mensalmente informações agregadas das operações de crédito recebidas através do Sistema de Informações de Créditos – SCR. Os relatórios são atualizados no último dia útil do mês, disponibilizando os dados após 60 dias do fechamento de cada período. São disponibilizados dados agregados de Carteira Ativa, Inadimplência e Ativo Problemático com possibilidade de detalhamento por tipo de cliente (PF/PJ), modalidade de crédito, unidade da federação, Classificação Nacional de Atividades Econômicas – CNAE (PJ), natureza da ocupação (PF), porte/rendimento dos clientes, origem de recursos e indexador das operações. Adicionalmente ao recurso gráfico apresentado na página, o relatório também disponibiliza aproximadamente 700 mil séries mensais acessíveis pelo portal de dados abertos. A publicação compreende o período de junho de 2012 até a atualidade. (Fonte: SCR.data - Painel de Operações de Crédito - Metodologia - https://dadosabertos.bcb.gov.br/dataset/scr_data/resource/6ed8c917-aace-4906-a8c4-09dd736e6872)

## **1.1. Definição do Problema**

O objetivo deste projeto é avaliar a evolução do endividamento das pessoas físicas no Brasil, no ano de 2022, conforme registro feito pelas instituições financeiras e comunicados ao Banco Central através do Sistema de Informações de Créditos (SCR). 

Fonte: 

### **1.2. Problema de aprendizado supervisionado ou não supervisionado?**

Não temos rótulos históricos dos dados e, por isso, não temos resultados esperados. Desta forma, podemos considerar como um problema de aprendizado não supervisionado. Tentaremos explorar os dados e aprender os possíveis relacionamentos entre eles.

### **1.3. Hipóteses sobre o problema**

Atuando no setor bancário/financeiro, pude perceber um intenso uso de crédito nos últimos anos. Num cenário de inflação alta (IPCA de 10,06% em 2021 e 5,79% em 2022), muitas vezes as famílias recorrem à contratação de operações de crédito para pagar despesas recorrentes. 

Além disso, a elevação da margem (30% para 35% e depois para 40% da renda bruta) para contratação de crédito consignado em alguns convênios como INSS e servidores federais, alavancou a concessão de crédito, ainda que as taxas não estivessem atrativas.

A hipótese que vou analisar é se houve uma elevação real no endividamento geral das famílias e se houve elevação da inadimplência.

## **2. Importação de pacotes, bibliotecas e carregamento dos dados**

### **2.1. Importação de pacotes e bibliotecas**

In [81]:
# Importando bibliotecas e pacotes que poderão ser utilizados na análise



# Importação de pacotes
import pandas as pd
import numpy as np
import glob
import seaborn as sns

# para tratamento de missings
import missingno as ms 

# Configuração para não exibir os warnings
import warnings
warnings.filterwarnings("ignore")

# Para tratamento de datas
from datetime import datetime

## Transformações Numéricas
from sklearn.preprocessing import MinMaxScaler # para normalização
from sklearn.preprocessing import StandardScaler # para padronização
## Transformações Categóricas
from sklearn.preprocessing import OneHotEncoder  # one-hot encoding e dummy encoding
from sklearn.preprocessing import OrdinalEncoder # ordinal encoding

In [82]:
# Versão do Pandas utilizada

pd.__version__

'1.5.3'

## **2.2. Carregamento dos dados**

### **2.2.1. Restrições ou condições para seleção dos dados**

Os dados do SCR foram obtidos no Portal de Dados Abertos do Banco Central do Brasil (https://dadosabertos.bcb.gov.br/). As informações são disponibilizadas desde 2012 mas, num primeiro momento, optei por trabalhar apenas com o ano de 2022.

Inicialmente, os dados analisados continham informações completas de clientes PF (Pessoas Físicas) e PJ (Pessoas Jurídicas) e o segmento das instituições(SR  - Segmento Resolução nº 4.553/2017), que variava de S1 a S5.

Devido ao tamanho dos arquivos (acima de 250MB cada base mensal), foi necessário reduzir a base analisada e foram excluídos ainda no arquivo *.csv* todas operações de clientes PJ e utilizado apenas informações de instituições S1.

*S1 - Bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas que tenham porte (Exposição/Produto Interno Bruto) superior a 10% ou exerçam atividade internacional relevante (ativos no exterior superiores a US$ 10 bilhões)*

### **2.2.2. Importando e unificando os dados**

In [83]:
# Cada arquivo .csv representa um mês da base de dados. Serão importados individualmente

url_2022_01 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202201.csv?raw=true', sep=';') # Base Janeiro/2022
url_2022_02 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202202.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Fevereiro/2022
url_2022_03 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202203.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Março/2022
url_2022_04 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202204.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Abril/2022
url_2022_05 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202205.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Maio/2022
url_2022_06 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202206.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Junho/2022
url_2022_07 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202207.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Julho/2022
url_2022_08 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202208.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Agosto/2022
url_2022_09 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202209.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Setembro/2022
url_2022_10 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202210.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Outubro/2022
url_2022_11 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202211.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Novembro/2022
url_2022_12 = pd.read_csv('https://github.com/simonefmr/datascience_mvp/blob/main/database_202212.csv?raw=true', sep=';', encoding='ISO-8859-1') # Base Dezembro/2022


In [84]:
colunas = ['data_base', 'uf', 'tcb', 'sr', 'cliente', 'ocupacao', 'cnae_secao','cnae_subclasse', 'porte', 'modalidade', 'origem', 'indexador', 'numero_de_operacoes', 'a_vencer_ate_90_dias', 'a_vencer_de_91_ate_360_dias', 'a_vencer_de_361_ate_1080_dias', 'a_vencer_de_1081_ate_1800_dias', 'a_vencer_de_1801_ate_5400_dias', 'a_vencer_acima_de_5400_dias', 'vencido_acima_de_15_dias', 'carteira_ativa', 'carteira_inadimplida_arrastada', 'ativo_problematico']

In [85]:
# Unificando os dados em um só dataset

scr_2022 = pd.concat(([url_2022_01, url_2022_02, url_2022_03, url_2022_04, url_2022_05, url_2022_06, url_2022_07, url_2022_08, url_2022_09, url_2022_10, url_2022_11, url_2022_12]), names='colunas')

### **2.2.:3. Definindo os atributos do dataset**



In [86]:
# Verificando as primeiras linhas

# Os dados foram carregados corretamente e estão classificados por database e depois por estado (UF) em ordem alfabética

scr_2022.head(5)

,data_base,uf,tcb,sr,cliente,ocupacao,cnae_secao,cnae_subclasse,porte,modalidade,...,a_vencer_ate_90_dias,a_vencer_de_91_ate_360_dias,a_vencer_de_361_ate_1080_dias,a_vencer_de_1081_ate_1800_dias,a_vencer_de_1801_ate_5400_dias,a_vencer_acima_de_5400_dias,vencido_acima_de_15_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
0,31/01/2022,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,...,"0,00","5561,30","0,00","0,00","0,00","0,00","0,00","5561,30","0,00","0,00"
1,31/01/2022,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,...,"3791034,42","982378,88","74016,25","9006,30","3741,61","0,00","281036,64","5141214,10","176044,79","237272,93"
2,31/01/2022,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,...,"1764710,58","4456712,22","11316762,96","9822143,26","12026778,46","0,00","306584,87","39693692,35","1208264,90","1278420,45"
3,31/01/2022,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Empréstimo sem consignação em folha,...,"293111,91","888394,40","1134747,73","636850,19","286267,62","0,00","1054,48","3240426,33","0,00","131338,05"
4,31/01/2022,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Habitacional,...,"140231,05","384054,93","867701,21","678425,45","1528340,60","353735,40","0,00","3952488,64","0,00","102216,54"


In [87]:
# Verificando as últimas linhas observamos a database de Dezembro (última cronologicamente) e seguindo a ordem alfabética das UF's.

scr_2022.tail(5)

,data_base,uf,tcb,sr,cliente,ocupacao,cnae_secao,cnae_subclasse,porte,modalidade,...,a_vencer_ate_90_dias,a_vencer_de_91_ate_360_dias,a_vencer_de_361_ate_1080_dias,a_vencer_de_1081_ate_1800_dias,a_vencer_de_1801_ate_5400_dias,a_vencer_acima_de_5400_dias,vencido_acima_de_15_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
34684,31/12/2022,TO,Bancário,S1,PF,PF - Servidor ou empregado público,-,-,PF - Sem rendimento,PF - Cartão de crédito,...,"19519,07","3862,50","0,00","0,00","0,00","0,00","63499,23","86880,80","59662,51","59716,09"
34685,31/12/2022,TO,Bancário,S1,PF,PF - Servidor ou empregado público,-,-,PF - Sem rendimento,PF - Empréstimo com consignação em folha,...,"12487,31","33544,11","36794,48","3700,46","456,58","0,00","13523,80","100506,74","15452,51","16351,70"
34686,31/12/2022,TO,Bancário,S1,PF,PF - Servidor ou empregado público,-,-,PF - Sem rendimento,PF - Empréstimo sem consignação em folha,...,"2854,12","6740,12","9479,85","1238,55","0,00","0,00","0,00","20312,64","0,00","0,00"
34687,31/12/2022,TO,Bancário,S1,PF,PF - Servidor ou empregado público,-,-,PF - Sem rendimento,PF - Outros créditos,...,"18,33","0,00","0,00","0,00","0,00","0,00","0,00","18,33","0,00","0,00"
34688,31/12/2022,TO,Bancário,S1,PF,PF - Servidor ou empregado público,-,-,PF - Sem rendimento,PF - Veículos,...,"2830,24","7595,95","14456,94","2850,96","0,00","0,00","0,00","27734,09","0,00","0,00"


## **3. Análise dos Dados**

In [88]:
# Verificando dimensões do dataset (quantos atributos e instâncias existem)

scr_2022.shape

(416856, 23)

In [89]:
# Resumo estatístico dos atributos com valor numérico.

scr_2022.describe()

,data_base,uf,tcb,sr,cliente,ocupacao,cnae_secao,cnae_subclasse,porte,modalidade,...,a_vencer_ate_90_dias,a_vencer_de_91_ate_360_dias,a_vencer_de_361_ate_1080_dias,a_vencer_de_1081_ate_1800_dias,a_vencer_de_1801_ate_5400_dias,a_vencer_acima_de_5400_dias,vencido_acima_de_15_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
count,416856,416856,416856,412963,416856,416856,416856,416856,416856,416856,...,416856,416856,416856,416856,416856,416856,416856,416856,416856,416856
unique,12,27,2,1,1,8,1,1,9,7,...,357248,362196,344336,299917,278181,94648,288423,407788,237230,272559
top,30/11/2022,SP,Bancário,S1,PF,PF - Outros,-,-,PF - Mais de 5 a 10 salários mínimos,PF - Outros créditos,...,"0,00","0,00","0,00","0,00","0,00","0,00","0,00","250,00","0,00","0,00"
freq,38621,22636,412963,412963,416856,64464,416856,416856,57213,96596,...,46745,46236,61695,101252,126312,295631,104336,66,152144,120373


Neste ponto, o retorno não foi o esperado. Não foram apresentados valores numéricos representativos. Apenas apresentadas contagens da quantidade de instâncias.

In [90]:
# Quais são os tipos de atributos que foram carregados?

scr_2022.dtypes       # Neste ponto podemos perceber que todos os atributos estão 
                      # classificados como "object" (string/texto). Será necessário 
                      # converter as instâncias para realizar as análises corretamente.

data_base                         object
uf                                object
tcb                               object
sr                                object
cliente                           object
ocupacao                          object
cnae_secao                        object
cnae_subclasse                    object
porte                             object
modalidade                        object
origem                            object
indexador                         object
numero_de_operacoes               object
a_vencer_ate_90_dias              object
a_vencer_de_91_ate_360_dias       object
a_vencer_de_361_ate_1080_dias     object
a_vencer_de_1081_ate_1800_dias    object
a_vencer_de_1801_ate_5400_dias    object
a_vencer_acima_de_5400_dias       object
vencido_acima_de_15_dias          object
carteira_ativa                    object
carteira_inadimplida_arrastada    object
ativo_problematico                object
dtype: object

### **3.1 Definindo os Atributos**

A definição dos atributos é feita conforme normativos do Banco Central, Receita Federal e IBGE, sendo informados no manual de Painel de Crédito - metodologia do SCR.

**'data_base'**: Data da apuração dos saldos de endividamento e repassado pelas instituições financeiras ao Banco Central.

**'uf'**: A informação segregada por unidades da federação é baseada no CEP de residência das pessoas físicas ou da Sede das pessoas jurídicas.

**'tcb'**: Tipo de Consolidado Bancário. Classificado em Bancário (Bancos comerciais, múltiplos, caixas econômicas, banco de câmbio etc), Não bancário (Não bancário de crédito, Não bancário de mercado de capitais e instituiçõe de pagamento) ou Cooperativas.

**'sr'**: Classificação conforme Segmento Resolução nº 4.553/2017, se refere ao porte da instituição. Neste MVP utilizarei apenas a segmentação S1, que trata de Bancos múltiplos, bancos comerciais, bancos de investimento, bancos de câmbio e caixas econômicas que tenham porte (Exposição/Produto Interno Bruto) superior a 10% ou exerçam atividade internacional relevante (ativos no exterior superiores a US$ 10 bilhões).

**'cliente'**: Pessoa Física ou Pessoa Jurídica (utilizarei apenas a base de PF)

**'ocupacao'**: Natureza da ocupação das pessoas físicas, podendo ser servidor ou empregado público, empregado de entidades sem fins lucrativos, empregado de empresa privada, aposentado/pensionista, autônomo, empresário, MEI ou outros.

**'cnae_secao'**: Código Nacional de Atividades Econômicos - CNAE (Pessoas Jurídicas). Busca padronizar a forma com que os diferentes órgãos do País classificam as unidades econômicas segundo suas atividades, refere às atividades exercidas por uma pessoa jurídica. Está informação será desconsiderada neste projeto.

**'cnae_subclasse'**: As subclasses segmentam ainda mais a atividade, informando e especificando o que cada unidade (matriz ou filial) produz. Também será desconsiderada pois não trataremos dados de PJ.

**'porte'**: Porte está relacionado à renda das pessoas físicas (em salários mínimos) ou ao porte de pessoa jurídica (Micro, Pequeno, Médio ou Grande), definido pela receita com base na receita bruta anual de cada empresa.

**'modalidade'**: Tipo de crédito contratado (cartão de crédito, empréstimo com ou sem consignação em folha), habitacional, rural, veículos etc.

**'origem'**: Se o recurso tem ou não destinação específica

**'indexador'**: Prefixado, Pós-fixado, Flutuantes, Índices de preços, TCR/TRFC ou Outros indexadores.

**'numero_de_operacoes'**: Número de operações de crédito para uma dada série. Abaixo de 15 operações, o relatório apresenta a informação "<= 15".

**'a_vencer_ate_90_dias' (a)**: Somatório dos créditos a vencer até 90 dias.

**'a_vencer_de_91_ate_360_dias' (b)**: Somatório dos créditos a vencer de 91 a 360 dias.

**'a_vencer_de_361_ate_1080_dias' (c)**: Somatório dos créditos a vencer de 361 a 1080 dias.

**'a_vencer_de_1081_ate_1800_dias' (d)**: Somatório dos créditos a vencer de 1081 até 1800 dias.

**'a_vencer_de_1801_ate_5400_dias' (e)**: Somatório dos créditos a vencer de 1801 a 5400 dias.

**'a_vencer_acima_de_5400_dias' (f)**: Somatório dos créditos a vencer acima de 5400 dias

**'vencido_acima_de_15_dias'**: Somatório de créditos vencidos acima de 15 dias

**'carteira_ativa'**: Somatório de todas as operações a vencer (a, b, c, d, e, f).

**'carteira_inadimplida_arrastada'**: Somatório das operações de crédito a vencer e vencidos que possuam alguma parcela vencida há mais de 90 dias.

**'ativo_problematico'**: São consideradas como ativos problemáticos as operações de crédito em atraso há mais de noventa dias e as operações nas quais existem indícios de que respectiva obrigação não será integralmente honrada. Entende-se que há indicativos de que a obrigação poderá não ser integralmente honrada, entre outros eventos, quando a operação for objeto de reestruturação e a instituição financeira reconhecer contabilmente deterioração significativa da 
qualidade do crédito do tomador, classificando-o entre os níveis de risco E e H.

### **3.2 Pré-processamento de dados**

In [91]:
# Na coluna de Número de Operações, vou desconsiderar as instituições que possuem <= 15 operações por serem muito pouco expressivas em relação ao restante dos dados

scr_2022 = scr_2022[scr_2022.numero_de_operacoes != '<= 15']
scr_2022.head(5)

,data_base,uf,tcb,sr,cliente,ocupacao,cnae_secao,cnae_subclasse,porte,modalidade,...,a_vencer_ate_90_dias,a_vencer_de_91_ate_360_dias,a_vencer_de_361_ate_1080_dias,a_vencer_de_1081_ate_1800_dias,a_vencer_de_1801_ate_5400_dias,a_vencer_acima_de_5400_dias,vencido_acima_de_15_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
1,31/01/2022,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,...,"3791034,42","982378,88","74016,25","9006,30","3741,61","0,00","281036,64","5141214,10","176044,79","237272,93"
2,31/01/2022,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,...,"1764710,58","4456712,22","11316762,96","9822143,26","12026778,46","0,00","306584,87","39693692,35","1208264,90","1278420,45"
3,31/01/2022,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Empréstimo sem consignação em folha,...,"293111,91","888394,40","1134747,73","636850,19","286267,62","0,00","1054,48","3240426,33","0,00","131338,05"
4,31/01/2022,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Habitacional,...,"140231,05","384054,93","867701,21","678425,45","1528340,60","353735,40","0,00","3952488,64","0,00","102216,54"
5,31/01/2022,AC,Bancário,S1,PF,PF - Aposentado/pensionista,-,-,PF - Acima de 20 salários mínimos,PF - Outros créditos,...,"111136,85","204003,84","129116,57","65424,61","32518,41","0,00","27261,51","569461,79","24239,26","121027,64"


In [92]:
# Colunas a serem excluídas:
# Coluna que especifica o TCB pois todos analisados serão do tipo "Bancário";
# A coluna SR, pois utilizaremos apenas instituições do tipo "S1";
# A coluna de tipo de cliente pois já selecionamos apenas os do tipo "PF";
# E as colunas de CNAE pois se referem ao enquadramento de empresas PJ.

scr_2022.drop(['tcb', 'sr', 'cliente', 'cnae_secao','cnae_subclasse'], axis=1, inplace=True)

In [104]:
scr_2022['data_base'] = pd.to_datetime(scr_2022['data_base'], format='%d/%m/%Y')

In [106]:
# Convertendo Coluna Object em Int

scr_2022['numero_de_operacoes'] = scr_2022['numero_de_operacoes'].astype(int)
scr_2022.head(5)

,data_base,uf,ocupacao,porte,modalidade,origem,indexador,numero_de_operacoes,a_vencer_ate_90_dias,a_vencer_de_91_ate_360_dias,a_vencer_de_361_ate_1080_dias,a_vencer_de_1081_ate_1800_dias,a_vencer_de_1801_ate_5400_dias,a_vencer_acima_de_5400_dias,vencido_acima_de_15_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
1,2022-01-31,AC,PF - Aposentado/pensionista,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,Sem destinação específica,Prefixado,708,3791034.42,982378.88,74016.25,9006.30,3741.61,0.0,281036.64,5141214.10,176044.79,237272.93
2,2022-01-31,AC,PF - Aposentado/pensionista,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,Sem destinação específica,Prefixado,715,1764710.58,4456712.22,11316762.96,9822143.26,12026778.46,0.0,306584.87,39693692.35,1208264.90,1278420.45
3,2022-01-31,AC,PF - Aposentado/pensionista,PF - Acima de 20 salários mínimos,PF - Empréstimo sem consignação em folha,Sem destinação específica,Prefixado,142,293111.91,888394.40,1134747.73,636850.19,286267.62,0.0,1054.48,3240426.33,0.00,131338.05
4,2022-01-31,AC,PF - Aposentado/pensionista,PF - Acima de 20 salários mínimos,PF - Habitacional,Com destinação específica,Pós-fixado,23,140231.05,384054.93,867701.21,678425.45,1528340.60,353735.4,0.00,3952488.64,0.00,102216.54
5,2022-01-31,AC,PF - Aposentado/pensionista,PF - Acima de 20 salários mínimos,PF - Outros créditos,Sem destinação específica,Prefixado,106,111136.85,204003.84,129116.57,65424.61,32518.41,0.0,27261.51,569461.79,24239.26,121027.64


In [110]:
# Altera o separador

scr_2022['a_vencer_ate_90_dias']            = scr_2022['a_vencer_ate_90_dias'].str.replace('.','').str.replace(',','.')
scr_2022['a_vencer_de_91_ate_360_dias']     = scr_2022['a_vencer_de_91_ate_360_dias'].str.replace('.','').str.replace(',','.')
scr_2022['a_vencer_de_361_ate_1080_dias']   = scr_2022['a_vencer_de_361_ate_1080_dias'].str.replace('.','').str.replace(',','.')
scr_2022['a_vencer_de_1081_ate_1800_dias']  = scr_2022['a_vencer_de_1081_ate_1800_dias'].str.replace('.','').str.replace(',','.')
scr_2022['a_vencer_de_1801_ate_5400_dias']  = scr_2022['a_vencer_de_1801_ate_5400_dias'].str.replace('.','').str.replace(',','.')
scr_2022['a_vencer_acima_de_5400_dias']     = scr_2022['a_vencer_acima_de_5400_dias'].str.replace('.','').str.replace(',','.')
scr_2022['vencido_acima_de_15_dias']        = scr_2022['vencido_acima_de_15_dias'].str.replace('.','').str.replace(',','.')
scr_2022['carteira_ativa']                  = scr_2022['carteira_ativa'].str.replace('.','').str.replace(',','.')
scr_2022['carteira_inadimplida_arrastada']  = scr_2022['carteira_inadimplida_arrastada'].str.replace('.','').str.replace(',','.')
scr_2022['ativo_problematico']              = scr_2022['ativo_problematico'].str.replace('.','').str.replace(',','.')

AttributeError: ignored

In [ ]:
scr_2022.reset_index(drop=True)

In [109]:
# Convertendo tipo de dado

scr_2022['a_vencer_ate_90_dias']                = scr_2022['a_vencer_ate_90_dias'].astype(dtype ='float')
scr_2022['a_vencer_de_91_ate_360_dias']         = scr_2022['a_vencer_de_91_ate_360_dias'].astype(dtype ='float')
scr_2022['a_vencer_de_361_ate_1080_dias']       = scr_2022['a_vencer_de_361_ate_1080_dias'].astype(dtype ='float')
scr_2022['a_vencer_de_1081_ate_1800_dias']      = scr_2022['a_vencer_de_1081_ate_1800_dias'].astype(dtype ='float')
scr_2022['a_vencer_de_1801_ate_5400_dias']      = scr_2022['a_vencer_de_1801_ate_5400_dias'].astype(dtype ='float')
scr_2022['a_vencer_acima_de_5400_dias']         = scr_2022['a_vencer_acima_de_5400_dias'].astype(dtype ='float')
scr_2022['vencido_acima_de_15_dias']            = scr_2022['vencido_acima_de_15_dias'].astype(dtype ='float')
scr_2022['carteira_ativa']                      = scr_2022['carteira_ativa'].astype(dtype ='float')
scr_2022['carteira_inadimplida_arrastada']      = scr_2022['carteira_inadimplida_arrastada'].astype(dtype ='float')
scr_2022['ativo_problematico']                  = scr_2022['ativo_problematico'].astype(dtype ='float')

scr_2022.head(5)

,data_base,uf,ocupacao,porte,modalidade,origem,indexador,numero_de_operacoes,a_vencer_ate_90_dias,a_vencer_de_91_ate_360_dias,a_vencer_de_361_ate_1080_dias,a_vencer_de_1081_ate_1800_dias,a_vencer_de_1801_ate_5400_dias,a_vencer_acima_de_5400_dias,vencido_acima_de_15_dias,carteira_ativa,carteira_inadimplida_arrastada,ativo_problematico
1,2022-01-31,AC,PF - Aposentado/pensionista,PF - Acima de 20 salários mínimos,PF - Cartão de crédito,Sem destinação específica,Prefixado,708,3791034.42,982378.88,74016.25,9006.30,3741.61,0.0,281036.64,5141214.10,176044.79,237272.93
2,2022-01-31,AC,PF - Aposentado/pensionista,PF - Acima de 20 salários mínimos,PF - Empréstimo com consignação em folha,Sem destinação específica,Prefixado,715,1764710.58,4456712.22,11316762.96,9822143.26,12026778.46,0.0,306584.87,39693692.35,1208264.90,1278420.45
3,2022-01-31,AC,PF - Aposentado/pensionista,PF - Acima de 20 salários mínimos,PF - Empréstimo sem consignação em folha,Sem destinação específica,Prefixado,142,293111.91,888394.40,1134747.73,636850.19,286267.62,0.0,1054.48,3240426.33,0.00,131338.05
4,2022-01-31,AC,PF - Aposentado/pensionista,PF - Acima de 20 salários mínimos,PF - Habitacional,Com destinação específica,Pós-fixado,23,140231.05,384054.93,867701.21,678425.45,1528340.60,353735.4,0.00,3952488.64,0.00,102216.54
5,2022-01-31,AC,PF - Aposentado/pensionista,PF - Acima de 20 salários mínimos,PF - Outros créditos,Sem destinação específica,Prefixado,106,111136.85,204003.84,129116.57,65424.61,32518.41,0.0,27261.51,569461.79,24239.26,121027.64


In [ ]:
# Quais são os tipos de dados dos atributos?
# Após as alterações de separador e uso dos comandos "astype" podemos fazer a 
# análise com variáveis numéricas 

scr_2022.dtypes

### **3.3 Estatísticas descritivas**

In [ ]:
# Verificando dimensões do dataset após exclusão de colunas e linhas de números
# de operações <= 15

# Podemos observar a redução de 23 para 18 colunas e de 416856 para 240723 linhas

scr_2022.shape

Podemos observar que todos os atributos estão classificados como sendo do tipo "object". Será necessário fazer a conversão de algumas colunas para permitirmos a análise descritiva.

In [ ]:
# Tratamento de missings

scr_2022.info()

Neste dataset encontramos diversos atributos zerados mas eles não são dados faltantes (NaN). Verificamos que a contagem de não-nulos é igual à quantidade de linhas observada nas dimensões do dataset (240723).

In [ ]:
# Verificando se existem dados faltantes, discrepantes ou inconsistentes

scr_2022.isnull().sum()

In [ ]:
# Exibindo visualização matricial que evidencia a ausência de nulidade do dataset
# Neste caso não será preciso fazer ajustes de faltantes

ms.matrix(scr_2022)


Não se verifica valores faltantes ou inconsistes. As discrepâncias são esperadas tendo em vista que cada linha representa um tipo de operação ou destinação específicos e a renda do tomador também influencia o volume disponibilizado. Além disso, o total de clientes varia muito de uma instituição para outra, impactando a quantidade de operações realizadas.

In [ ]:
mediana = np.median(scr_2022['a_vencer_ate_90_dias'])
print(mediana)

In [ ]:
 media = np.mean(data) desv_pad = np.std(data) variancia = np.var(data)

### **5.3 Normalização e Padronização**

In [ ]:
# Faz um resumo estatístico do dataset (média, desvio padrão, mínimo, máximo e os quartis)
scr_2022.describe()

In [ ]:
# Correlação de Pearson
scr_2022.corr(method = 'pearson')

Estatísticas descritivas: 

Quantos atributos e instâncias existem?
Quais são os tipos de dados dos atributos?
Verifique as primeiras linhas do dataset. Algo chama a atenção?
Há valores faltantes, discrepantes ou inconsistentes?
Faça um resumo estatístico dos atributos com valor numérico (mínimo, máximo, mediana, moda, média, desvio-padrão e número de valores ausentes). O que você percebe?
Visualizações: 

Verifique a distribuição de cada atributo. O que você percebe? Dica: esta etapa pode dar ideias sobre a necessidade de transformações na etapa de preparação de dados (por exemplo, converter atributos de um tipo para outro, realizar operações de discretização, normalização, padronização, etc.).
Se for um problema de classificação, verifique a distribuição de frequência das classes. O que você percebe? Dica: esta etapa pode indicar a possível necessidade futura de balanceamento de classes.
Analise os atributos individualmente ou de forma combinada, usando os gráficos mais apropriados. 

Pré-processamento de dados: 
Objetivo: realizar operações de limpeza, tratamento e preparação dos dados. 

Verifique quais operações de pré-processamento podem ser interessantes para o seu problema e salve visões diferentes do seu dataset (por exemplo, normalização, padronização, discretização e one-hot-encoding).
Trate (removendo ou substituindo) os valores faltantes (se existentes).
Realize outras transformações de dados porventura necessárias.
Explique, passo a passo, as operações realizadas, justificando cada uma delas.
Se julgar necessário, utilizando os dados pré-processados, volte na etapa de análise exploratória e verifique se surge algum insight diferente após as operações realizadas.


# Calcular a média aritmétia, mediana, variância e desvio-padrão
mediana = np.median(data)
media = np.mean(data)
desv_pad = np.std(data)
variancia = np.var(data)

# Mostrar os resultados
print('Mediana: ', mediana)
print('Média: ', media)
print('Desvio Padrão: ', desv_pad)
print('Variância: ', variancia)

# Plotar Histograma
plt.hist(data, bins=5)
plt.xlabel('Salários')
plt.ylabel('Frequência')
plt.title('Distribuição Salarial')
plt.show()